# 1D FEM with Non-uniform Mesh

Solve the one-dimensional Poisson equation using linear finite elements:

$$
- \frac{d^2 u}{dx^2} = f(x), \qquad x\in[0,1], \qquad u(0)=u(1)=0
$$

- Choose a load (forcing) function $f(x)$ of your preference.
- Random sampling: generate $n$ interior nodes with random positions, include the boundary nodes $x_0 = 0$ and $x_{n+1} = 1$, and sort them in ascending order. Each element $e$ has length $h_e = x_{i+1} - x_i$.
- Plot the numerical solution $u(x)$, the load $f(x)$, and the distribution of the random nodes.

In [1]:
import numpy as np
import matplotlib.pyplot as plt